# controller

> Defines the interface between View & Model.

In [ ]:
#| default_exp controller

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
#| export
from neuralactivitycubic.model import Model
from neuralactivitycubic.view import WidgetsInterface
from neuralactivitycubic import results

In [ ]:
#|export
class App:

    def __init__(self):
        self.view = WidgetsInterface()
        self._bind_buttons_of_view_to_functions_of_model()
        self.pixel_conversion = 1/plt.rcParams['figure.dpi']


    def _setup_interaction_between_model_and_view(self) -> None:
        self.model.setup_connection_to_update_infos_in_view(self.view.update_infos)
        self.model.setup_connection_to_display_results(self.view.main_screen.show_output_screen, self.view.main_screen.output, self.pixel_conversion)


    def _bind_buttons_of_view_to_functions_of_model(self) -> None:
        self.view.source_data_panel.load_source_data_button.on_click(self._load_data_button_clicked)
        self.view.analysis_settings_panel.run_analysis_button.on_click(self._run_button_clicked)
        self.view.analysis_settings_panel.preview_window_size_button.on_click(self._preview_window_size_button_clicked)


    def launch(self) -> None:
        display(self.view.widget)


    def _load_data_button_clicked(self, change) -> None:
        self.model = Model(self.view.export_user_settings())
        self._setup_interaction_between_model_and_view()
        self.model.create_analysis_jobs()
        if len(self.model.analysis_job_queue) < 1:
            self.model.add_info_to_logs('Failed to create any analysis job(s). Please inspect logs for more details!', True)
            self.view.user_info_panel.progress_bar.bar_style = 'danger'
        else:
            self._display_preview_of_representative_job(self.view.grid_size)
            self.model.add_info_to_logs(f'Data import completed! {len(self.model.analysis_job_queue)} job(s) in queue.', True, 100.0)
            self.view.enable_analysis(True)


    def _display_preview_of_representative_job(self, grid_size: int) -> None:
        representative_job = self.model.analysis_job_queue[0]
        representative_job.load_data_into_memory(grid_size)
        self.view.adjust_widgets_to_loaded_data(total_frames = representative_job.recording.zstack.shape[0])
        self.view.main_screen.show_output_screen()
        with self.view.main_screen.output:
            fig = plt.figure(figsize = (600*self.pixel_conversion, 400*self.pixel_conversion))
            if representative_job.focus_area_enabled:
                results.plot_roi_boundaries(representative_job.focus_area, 'cyan', 'solid', 2)
            if representative_job.rois_source == 'file':
                for roi in representative_job.all_rois:
                    results.plot_roi_boundaries(roi, 'magenta', 'solid', 1)
            plt.imshow(representative_job.recording.preview, cmap = 'gray')
            plt.tight_layout()
            plt.show()


    def _run_button_clicked(self, change) -> None:
        self.view.enable_analysis(False)
        self.model.config = self.view.export_user_settings()
        self.model.run_analysis()
        self.model.add_info_to_logs(f'Processing of all jobs completed! Feel free to load more data & continue analyzing!', True, 100.0)
        self.view.enable_analysis(True)


    def _preview_window_size_button_clicked(self, change) -> None:
        self.view.main_screen.show_output_screen()
        with self.view.main_screen.output:
            preview_fig, preview_ax = self.model.preview_window_size(self.view.grid_size)
            preview_fig.set_figheight(400 * self.pixel_conversion)
            preview_fig.tight_layout()
            plt.show(preview_fig)

In [ ]:
#| export
def open_gui():
    """Start the interactive widgets interface for NeuralActivityCubic"""
    na3 = App()
    return na3.launch()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

# GUI Tests Dependencies

In [ ]:
from time import sleep
import ipykernel
from jupyter_server import serverapp
import requests
from pathlib import Path
from contextlib import contextmanager
from multiprocessing import Process


def get_kernel_id() -> str | None:
    try:
        connection_file = ipykernel.get_connection_file()
        return Path(connection_file).stem.split('-', 1).pop(-1)
    except RuntimeError:
        return None


def find_server_info_by_kernel_id(kernel_id: str):
    servers = list(serverapp.list_running_servers())
    for server in servers:
        try:
            response = requests.get(f"{server['url']}api/sessions", headers={'Authorization': f"token {server['token']}"})
            for session in response.json():
                if session['kernel']['id'] == kernel_id:
                    return server
        except Exception as e:
            continue

    return None


def find_server_info_by_process_id(process_id: int):
    servers = list(serverapp.list_running_servers())
    for server in servers:
        if server['pid'] == process_id:
            return server

    return None


def start_jupyter_server(project_root: Path):
    from jupyterlab.labapp import LabApp

    app = LabApp.initialize_server(argv=[
        f'--ServerApp.root_dir={project_root}',
        '--ServerApp.open_browser=false',
    ], no_browser_open_file=True)

    app.start()


@contextmanager
def run_jupyter_server_app(project_root: str):
    jupyter_process = Process(target=start_jupyter_server, args=(project_root,))
    jupyter_process.start()

    sleep(3)  # Wait for the server to start

    yield find_server_info_by_process_id(jupyter_process.pid)

    jupyter_process.terminate()


@contextmanager
def get_jupyter_server():
    kernel_id = get_kernel_id()

    if kernel_id:
        yield find_server_info_by_kernel_id(kernel_id)
    else:
        with run_jupyter_server_app(project_root=Path.cwd().parent) as server_info:
            yield server_info

In [ ]:
import uuid

from playwright.async_api import expect
from playwright.async_api import ViewportSize
from playwright.async_api import async_playwright
from jupyter_server import serverapp
from jupyter_server.services.contents.filemanager import FileContentsManager
from contextlib import contextmanager
from contextlib import asynccontextmanager
from nbformat import v4 as nbformat


@contextmanager
def create_temporary_notebook(cells: list[str]):
    fcm = FileContentsManager()
    name = f"test-{uuid.uuid4()}.ipynb"
    try:
        notebook = nbformat.new_notebook()
        for cell in cells:
            notebook['cells'].append(nbformat.new_code_cell(source=cell))
        fcm.save({'type': 'notebook', 'content': notebook}, name)
        yield name
    finally:
        fcm.delete(name)


@asynccontextmanager
async def provide_na3_gui(visible: bool = True):
    with get_jupyter_server() as server_info, create_temporary_notebook(cells=['import neuralactivitycubic as na3; na3.open_gui()']) as notebook_name:
        token = server_info['token']
        lab_url = server_info['url'] + 'lab/tree/nbs'

        async with async_playwright() as playwright:
            browser_type = playwright.chromium
            browser = await browser_type.launch(headless=not visible)
            page = await browser.new_page(viewport=ViewportSize(width=1280, height=1024))
            await page.goto(f'{lab_url}/{notebook_name}?token=' + token)
    
            await expect(page.get_by_role('main')).to_contain_text(notebook_name)
    
            selected_tab_id = await page.get_by_role('main').get_by_role('tab', selected=True).first.get_attribute('data-id')
            selected_tab = page.locator(f'#{selected_tab_id}').first
            await expect(selected_tab).to_be_visible()
    
            run_menu = page.get_by_role("menuitem").filter(has=page.get_by_text("Run")).first
            await run_menu.click()
    
            run_all_cells_option = page.get_by_role("menuitem").filter(has=page.get_by_text("Run All Cells")).first
            await expect(run_all_cells_option).to_be_visible()
            await run_all_cells_option.click()
            
            await page.keyboard.press("ControlOrMeta+B")
    
            na3_gui = selected_tab.locator('.box-na3-gui').first
            await expect(na3_gui).to_be_visible()
            
            yield na3_gui
    
            await browser.close()

In [ ]:
import os
visible = os.environ.get('CI') != 'true'

async with provide_na3_gui(visible=visible) as gui:
    general_settings = gui.locator('.box-general-settings')
    assert not await general_settings.get_by_role("button", name="Select").is_disabled()
    assert await general_settings.get_by_role("button", name="Load Data").is_disabled()

    analysis_settings = gui.locator('.box-analysis-settings')

    buttons = await analysis_settings.locator("button").all()
    assert buttons
    for button in buttons:
        assert await button.is_disabled()

    inputs = await analysis_settings.locator("input").all()
    assert inputs
    for input in inputs:
        assert await input.is_disabled()